# RoadRunner in pyPESTO

**After going through this notebook, you will be able to...**

* ... create a pyPESTO problem using [RoadRunner](https://www.libroadrunner.org) as a simulator directly from a PEtab problem.
* ... perform a parameter estimation using pyPESTO with RoadRunner as a simulator, setting advanced simulator features.

In [1]:
# install pyPESTO with roadrunner support
!pip install amici --quiet
!pip install pypesto[roadrunner,petab] --quiet

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for amici (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for amici
ERROR: Could not build wheels for amici, which is required to install pyproject.toml-based projects


In [2]:
# import
import amici
import matplotlib as mpl
import numpy as np
import petab
import pypesto.objective
import pypesto.optimize as optimize
import pypesto.objective.roadrunner as pypesto_rr
import pypesto.sample as sample
import pypesto.visualize as visualize
import pypesto.profile as profile
from IPython.display import Markdown, display
from pprint import pprint

mpl.rcParams["figure.dpi"] = 100
mpl.rcParams["font.size"] = 18

np.random.seed(1912)


# name of the model that will also be the name of the python module
model_name = "boehm_JProteomeRes2014"

# output directory
model_output_dir = "tmp/" + model_name

ModuleNotFoundError: No module named 'amici'

## Creating pyPESTO problem from PEtab

The [PEtab file format](https://petab.readthedocs.io/en/latest/documentation_data_format.html) stores all the necessary information to define a parameter estimation problem. This includes the model, the experimental data, the parameters to estimate, and the experimental conditions. Using the `pypesto_rr.PetabImporterRR` class, we can create a pyPESTO problem directly from a PEtab problem.

In [ ]:
petab_yaml = f"./{model_name}/{model_name}.yaml"

petab_problem = petab.Problem.from_yaml(petab_yaml)
importer = pypesto_rr.PetabImporterRR(petab_problem)
problem = importer.create_problem()

We now have a pyPESTO problem that we can use to perform parameter estimation. We can get some information on the RoadRunnerObjective and access the RoadRunner model.

In [ ]:
pprint(problem.objective.get_config())

In [ ]:
# direct simulation of the model using roadrunner
sim_res = problem.objective.roadrunner_instance.simulate(
    times=[0, 2.5, 5, 10, 20, 50]
)
pprint(sim_res)
problem.objective.roadrunner_instance.plot();

For more details on interacting with the roadrunner instance, we refer to the [documentation of libroadrunner](https://libroadrunner.readthedocs.io/en/latest/). However, we point out that while theoretical possible, we **strongly advice against** changing the model in that manner.

In [ ]:
ret = problem.objective(
    petab_problem.get_x_nominal(fixed=False,scaled=True),
    mode="mode_fun",
    return_dict=True,
)
pprint(ret)

## Optimization

To optimize a problem using a RoadRunner objective, we can set additional solver options for the ODE solver.

In [ ]:
optimizer = optimize.ScipyOptimizer()

solver_options = pypesto_rr.SolverOptions(
    relative_tolerance = 1e-6,
    absolute_tolerance = 1e-12,
    maximum_num_steps = 10000
)
engine = pypesto.engine.SingleCoreEngine()
problem.objective.solver_options = solver_options

In [ ]:
result = optimize.minimize(
    problem=problem,
    optimizer=optimizer,
    n_starts=5,  # usually a value >= 100 should be used
    engine=engine
)
display(Markdown(result.summary()))

Disclaimer: Currently there are two main things not yet fully supported with roadrunner objectives. One  is parallelization of the optimization using MultiProcessEngine. The other is explicit gradients of the objective function. While the former will be added in a near future version, we will show a workaround for the latter, until it is implemented.

### Visualization Methods

In order to visualize the optimization, there are a few things possible. For a more extensive explanation we refer to the "getting started" notebook.

In [ ]:
visualize.waterfall(result);

In [ ]:
visualize.parameters(result);

In [ ]:
visualize.parameters_correlation_matrix(result);

### Sensitivities via finite differences

Some solvers need a way to calculate the sensitivities, which currently RoadRunner Objectives do not suport. For this scenario, we can use the FiniteDifferences objective in pypesto, which wraps a given objective into one, that computes sensitivities via finite differences.

In [ ]:
# no support for sensitivities
try:
    ret = problem.objective(
        petab_problem.get_x_nominal(fixed=False,scaled=True),
        mode="mode_fun",
        return_dict=True,
        sensi_orders=(1,),
    )
    pprint(ret)
except ValueError as e:
    pprint(e)

In [ ]:
objective_fd = pypesto.objective.FD(problem.objective)
# support through finite differences
try:
    ret = objective_fd(
        petab_problem.get_x_nominal(fixed=False,scaled=True),
        mode="mode_fun",
        return_dict=True,
        sensi_orders=(1,),
    )
    pprint(ret)
except ValueError as e:
    pprint(e)